In [6]:
import pandas as pd

# Example: load 2023 season play-by-play

def show_columns(filename): 
    df = pd.read_csv(filename)
    
    # https://github.com/nflverse/nflverse-data/releases/tag/pbp
    
    print(df.columns)
    df.head()
    
    for column_name in df:
        print(f"{column_name}")


In [25]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

def load_data(filename): 
    
    DB_CONFIG = {
        "dbname": "nfl_historical",
        "user": "postgres",
        "password": "postgres",
        "host": "localhost",
        "port": 5432,
    }
    
    # === READ CSV ===
    df = pd.read_csv(filename)
    
    # === CONNECT TO DATABASE ===
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    
    # === CREATE TABLES IF NOT EXIST ===
    cur.execute("""
    CREATE TABLE IF NOT EXISTS games (
        id TEXT PRIMARY KEY,
        home_team TEXT NOT NULL,
        away_team TEXT NOT NULL,
        description TEXT NOT NULL,
        game_date date NOT NULL,
        season_type TEXT NOT NULL,
        week smallint NOT NULL
    );
    """)
    
    cur.execute("""
    CREATE TABLE IF NOT EXISTS plays (
        id TEXT PRIMARY KEY,
        game_id TEXT NOT NULL REFERENCES games(id) ON DELETE CASCADE,
        posteam TEXT NOT NULL,
        posteam_type TEXT NOT NULL,
        defteam TEXT NOT NULL,
        total_home_score int,
        total_away_score int,
        qtr smallint,
        quarter_seconds_remaining float,
        game_seconds_remaining float
    );
    """)
    
    conn.commit()
    
    # === INSERT INTO games ===
    # Use UPSERT to avoid duplicates (in case same game appears multiple times)
    games = df[["game_id", "home_team", "away_team", "desc", "game_date", "season_type", "week", ]].drop_duplicates(subset=["game_id"])
    
    execute_values(
        cur,
        """
        INSERT INTO games (id, home_team, away_team, description, game_date, season_type, week)
        VALUES %s
        ON CONFLICT (id) DO UPDATE SET 
          home_team = EXCLUDED.home_team,
          away_team = EXCLUDED.away_team,
          game_date = EXCLUDED.game_date,
          season_type = EXCLUDED.season_type,
          week = EXCLUDED.week
        """,
        list(games.itertuples(index=False, name=None))
    )
    
    # === INSERT INTO plays ===
    plays = df[["play_id", "game_id", "posteam", "posteam_type", "defteam", "total_home_score", "total_away_score", "qtr", "quarter_seconds_remaining", "game_seconds_remaining"]].drop_duplicates(subset=["play_id"])
    
    plays["play_id"] = plays["game_id"].astype(str) + "_" + plays["play_id"].astype(str)
    
    execute_values(
        cur,
        """
        INSERT INTO plays (
            id, 
            game_id, 
            posteam, 
            posteam_type, 
            defteam,
            total_home_score, 
            total_away_score, 
            qtr,
            quarter_seconds_remaining,
            game_seconds_remaining
        )
        VALUES %s
        ON CONFLICT (id) DO UPDATE SET
          game_id = EXCLUDED.game_id,
          posteam = EXCLUDED.posteam,
          posteam_type = EXCLUDED.posteam_type,
          defteam = EXCLUDED.defteam,
          total_home_score = EXCLUDED.total_home_score,
          total_away_score = EXCLUDED.total_away_score,
          qtr = EXCLUDED.qtr,
          quarter_seconds_remaining = EXCLUDED.game_seconds_remaining,
          game_seconds_remaining = EXCLUDED.game_seconds_remaining
        """,
        list(plays.itertuples(index=False, name=None))
    )
    
    # === COMMIT AND CLEANUP ===
    conn.commit()
    cur.close()
    conn.close()
    
    print("Data import completed successfully.")
    print(filename)


In [26]:

#show_columns("./play_by_play_2006.csv");

In [29]:
#load_data("./play_by_play_2006.csv")
#load_data("./play_by_play_2020.csv")
#load_data("./play_by_play_2021.csv")
load_data("../../data/nfl/play_by_play_2012.csv")
load_data("../../data/nfl/play_by_play_2013.csv")
load_data("../../data/nfl/play_by_play_2014.csv")
load_data("../../data/nfl/play_by_play_2015.csv")
load_data("../../data/nfl/play_by_play_2016.csv")
load_data("../../data/nfl/play_by_play_2017.csv")
load_data("../../data/nfl/play_by_play_2018.csv")
load_data("../../data/nfl/play_by_play_2019.csv")
load_data("../../data/nfl/play_by_play_2022.csv")
#load_data("./play_by_play_2023.csv")
#load_data("./play_by_play_2024.csv")

/tmp/ipykernel_47431/3253985518.py:16: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,213,214,218,219,220,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Data import completed successfully.
../../data/nfl/play_by_play_2012.csv


/tmp/ipykernel_47431/3253985518.py:16: DtypeWarning: Columns (179,180,182,183,193,194,197,198,203,204,205,206,209,210,218,219,220,253,254,255,260,262,263,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Data import completed successfully.
../../data/nfl/play_by_play_2013.csv


/tmp/ipykernel_47431/3253985518.py:16: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,218,219,220,248,249,253,254,255,260,262,263,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


...

Data import completed successfully.
../../data/nfl/play_by_play_2018.csv


/tmp/ipykernel_47431/3253985518.py:16: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,218,219,220,233,234,235,236,237,238,248,249,253,254,255,260,262,263,283,284,302) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Data import completed successfully.
../../data/nfl/play_by_play_2019.csv


/tmp/ipykernel_47431/3253985518.py:16: DtypeWarning: Columns (45,179,180,182,183,187,188,189,190,193,194,197,198,203,204,205,206,213,214,218,219,220,222,224,226,243,244,245,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Data import completed successfully.
../../data/nfl/play_by_play_2022.csv


In [35]:
import pandas as pd
import numpy as np
import psycopg2

def calculate_probabilities(): 
    conn = psycopg2.connect(
        host="localhost",        # or your host name
        port="5432",             # default Postgres port
        dbname="nfl_historical",  # change this
        user="postgres",        # change this
        password="postgres" # change this
    )
    
    games = pd.read_sql("SELECT * FROM games;", conn)
    plays = pd.read_sql("SELECT * FROM plays;", conn)
    
    print("Games:", games.shape)
    print("Plays:", plays.shape)
    print(games.head())
    print(plays.head())
    
    conn.close()

    #Add outcome (did home team win?) to each game ---
    games["home_win"] = (games["total_home_score"] > games["total_away_score"]).astype(int)
    
    #Merge plays with their game's win outcome ---
    df = plays.merge(games[["id", "home_win"]], left_on="game_id", right_on="id", how="inner")
    
    #Filter for 4th quarter only ---
    df = df[df["qtr"] == 4].copy()
    
    #Compute score differential (home - away) ---
    df["diff"] = df["total_home_score"] - df["total_away_score"]
    
    #Define time buckets (seconds remaining) ---
    def time_bucket(sec_left):
        if 60 <= sec_left < 120:
            return "1-2 min"
        elif 120 <= sec_left < 180:
            return "2-3 min"
        elif 180 <= sec_left < 240:
            return "3-4 min"
        elif 240 <= sec_left < 300:
            return "4-5 min"
        else:
            return None
    
    df["time_bucket"] = df["game_seconds_remaining"].apply(time_bucket)
    
    #Define score buckets ---
    def score_bucket(diff):
        if diff == 3:
            return "ahead by 3"
        elif diff == 4:
            return "ahead by 4"
        elif diff == 5:
            return "ahead by 5"
        elif diff == 6:
            return "ahead by 6"
        elif diff == 7:
            return "ahead by 7"
        elif diff > 7:
            return "ahead by >7"
        else:
            return None
    
    df["score_bucket"] = df["diff"].apply(score_bucket)
    
    #Filter only cases where home team is leading (diff >= 3) and valid buckets ---
    df = df[(df["diff"] >= 3) & df["time_bucket"].notna() & df["score_bucket"].notna()]
    
    #Aggregate: compute empirical probability ---
    result = (
        df.groupby(["time_bucket", "score_bucket"])
          .agg(
              situations=("game_id", "count"),
              prob_home_win=("home_win", "mean")
          )
          .reset_index()
          .sort_values(["time_bucket", "score_bucket"])
    )
    
    print(result)


In [36]:
calculate_probabilities()

/tmp/ipykernel_47431/2765686778.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  games = pd.read_sql("SELECT * FROM games;", conn)
/tmp/ipykernel_47431/2765686778.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  plays = pd.read_sql("SELECT * FROM plays;", conn)


Games: (3811, 7)
Plays: (22600, 10)
                id home_team away_team description   game_date season_type  \
0  2023_01_ARI_WAS       WAS       ARI        GAME  2023-09-10         REG   
1  2023_01_BUF_NYJ       NYJ       BUF        GAME  2023-09-11         REG   
2  2023_01_CAR_ATL       ATL       CAR        GAME  2023-09-10         REG   
3  2023_01_CIN_CLE       CLE       CIN        GAME  2023-09-10         REG   
4  2023_01_DAL_NYG       NYG       DAL        GAME  2023-09-10         REG   

   week  
0     1  
1     1  
2     1  
3     1  
4     1  
                    id         game_id posteam posteam_type defteam  \
0  2012_01_ATL_KC_2619  2012_01_ATL_KC     ATL         away      KC   
1  2012_01_ATL_KC_2643  2012_01_ATL_KC     ATL         away      KC   
2  2012_01_ATL_KC_2665  2012_01_ATL_KC     ATL         away      KC   
3  2012_01_ATL_KC_2684  2012_01_ATL_KC      KC         home     ATL   
4  2012_01_ATL_KC_2706  2012_01_ATL_KC      KC         home     ATL   

   total

KeyError: 'total_home_score'